In [1]:
from datetime import datetime

import time
import os
import sys
import itertools
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.time import Time
import astropy.units as u

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

config = tf.ConfigProto(
    log_device_placement=True
)
sess = tf.Session(config=config)

np.random.seed(42)

print(tf.__version__)

1.14.0


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
# load data and make some subsets for each wfs for inspection later
dfs = []
for y in ["2018_2019"]:
    dfs.append(pd.read_csv(f"../raw_data/{y}_wfs_loose.csv"))
data = pd.concat(dfs)
data['ut'] = pd.to_datetime(data.ut)
data['az'][data['az'] < 0.] += 360.

f9 = data[(data['wfs'] == 'newf9') | (data['wfs'] == 'oldf9')]
f5 = data[data['wfs'] == 'f5']
mmirs = data[data['wfs'] == 'mmirs']
bino = data[data['wfs'] == 'binospec']

# wrangle the times to add colums for mjd to look for trends over time and hour to look for nightly trends
raw_times = data['time']
times = Time(raw_times.values.tolist(), format='isot', scale='utc')
mjd = times.mjd
data['mjd'] = mjd.tolist()
data['hour'] = data['ut'].dt.hour

e_series = pd.read_csv("../halcoll/data/halcoll_temps.csv")

fixed = data.drop(columns=['ut']).set_index(pd.DatetimeIndex(data['time'], name='ut'))

e_series = e_series.set_index(pd.DatetimeIndex(e_series['ts'], name='ut').tz_localize('MST').tz_convert(None)).drop(columns=['ts'])

merged = pd.merge_asof(fixed.sort_index(), e_series, on='ut')

# trim out columns not relevant to training
dropped_cols = [
    'ut', 'time', 'airmass', 'cc_x_err', 'cc_y_err', 'chamt', 'osst',
    'outt', 'exptime', 'file', 'focerr', 'fwhm', 'raw_seeing', 'residual_rms',
    'seeing', 'wavefront_rms', 'xcen', 'ycen', 'comaerr'
]
trimmed = merged.drop(columns=dropped_cols)
trimmed = trimmed.dropna()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [4]:
def norm(x, s):
    return (x - s['mean']) / s['std']

labels = ['focus', 'tiltx', 'tilty', 'transx', 'transy']

# split data set up into three training sets: f9, mmirs, and f5/bino. these sets correspond to the three optical configurations
# we use: f/9 and no corrector, f/5 with no corrector (mmirs), and f/5 with spectroscopic corrector (f5/bino).
datasets = {}
wfs = trimmed.pop('wfs')
datasets['f9'] = trimmed[wfs == 'newf9']
datasets['mmirs'] = trimmed[wfs == 'mmirs']
datasets['f5'] = trimmed[(wfs == 'f5') | (wfs == 'binospec')]

# the large range in offsets is messing up the training. so normalize the hexapod coords to their means
means = {}
train_datasets = {}
test_datasets = {}
train_labels = {}
test_labels = {}
train_stats = {}
normed_train_data = {}
normed_test_data = {}
for w in datasets:
    means[w] = {}
    for l in labels:
        means[w][l] = datasets[w][l].mean()
        datasets[w][l] -= means[w][l]

    train_datasets[w] = datasets[w].sample(frac=0.8, random_state=0)
    test_datasets[w] = datasets[w].drop(train_datasets[w].index)
    train_stats[w] = train_datasets[w].describe()
    train_stats[w] = train_stats[w].drop(columns=labels)
    train_stats[w] = train_stats[w].transpose()
    
    train_labels[w] = {}
    test_labels[w] = {}
    for l in labels:
        train_labels[w][l] = train_datasets[w].pop(l)
        test_labels[w][l] = test_datasets[w].pop(l)
        
    normed_train_data[w] = norm(train_datasets[w], train_stats[w])
    normed_test_data[w] = norm(test_datasets[w], train_stats[w])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
def build_model(width=32, shape=1, nlayers=1, tolerance=0.001):
    nn_layers = [layers.Dense(width, activation=tf.nn.relu, input_shape=[shape])]  # input_shape=[len(train_dataset.keys())])]

    if nlayers > 0:
        for i in range(nlayers-1):
            nn_layers.append(layers.Dense(width, activation=tf.nn.relu))
    
    nn_layers.append(layers.Dense(1))
    model = keras.Sequential(nn_layers)
    optimizer = tf.keras.optimizers.RMSprop(tolerance)

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    return model

In [6]:
models = {}
mod_config = {
    "f9": {
        'width': 8,
        'nlayers': 3
    },
    "f5": {
        'width': 64,
        'nlayers': 2
    },
    'mmirs': {
        'width': 64,
        'nlayers': 2
    }
}

for w in datasets:
    models[w] = {}
    for l in labels:
        models[w][l] = build_model(
            width=mod_config[w]['width'],
            shape=len(train_datasets[w].keys()),
            nlayers=mod_config[w]['nlayers']
        )

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

W0921 16:05:45.584440 139852920898944 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
models['mmirs']['focus'].summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 64)                1280      
_________________________________________________________________
dense_21 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 65        
Total params: 5,505
Trainable params: 5,505
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

EPOCHS = 4000

histories = {}

for l in labels:
    print(f"Training {l} model....")
    histories[l] = models['mmirs'][l].fit(
        normed_train_data['mmirs'], train_labels['mmirs'][l],
        epochs=EPOCHS, validation_split = 0.2, verbose=0,
        callbacks=[early_stop, PrintDot()]
    )
    print("\n")

Training focus model....

....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.................................................................

In [16]:
for l in labels:
    models['f9'][l].save(f"../halcoll/data/mmirs_{l}_2018_2019_loose_64x64_model.h5")

In [17]:
def plot_history(history, label=None):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    plt.legend()
    if label is not None:
        plt.title(label)

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    plt.legend()
    if label is not None:
        plt.title(label)
    plt.savefig(f"{label}_train.pdf")
    plt.show()

In [18]:
plot_history(histories['focus'], label="focus")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
def show_results(wfs, label):
    loss, mae, mse = models[wfs][label].evaluate(normed_test_data[wfs], test_labels[wfs][label], verbose=0)

    print("Testing set Mean Abs Error: {:5.2f} um".format(mae))
    print("Testing set RMS: {:5.2f} um".format(np.sqrt(mse)))

    test_predictions = models[wfs][label].predict(normed_test_data[wfs]).flatten()

    plt.figure()
    plt.scatter(test_labels[wfs][label], test_labels[wfs][label] - test_predictions)
    plt.xlabel('True Values [um]')
    plt.ylabel('Residuals [um]')
    minx, maxx = min(test_labels[wfs][label]), max(test_labels[wfs][label])
    plt.plot([minx, maxx], [0, 0])
    plt.show()

In [20]:
show_results('mmirs', 'focus')

Testing set Mean Abs Error: 12.03 um
Testing set RMS: 18.48 um


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
lab = 'focus'
test_predictions = models['f9'][lab].predict(normed_test_data['f9']).flatten()
diff = test_labels['f9'][lab] - test_predictions
diff.idxmax(), diff.idxmin()

(18649, 5502)

In [ ]:
test_labels[lab].idxmin()

In [52]:
test_datasets['f9'].loc[5502]

az                                       254.111081
el                                        65.569023
mjd                                    58301.277643
hour                                       6.000000
cell_e_series_backplate_C                 11.031100
cell_e_series_chamber_ambient_C           14.983700
cell_e_series_frontplate_C                11.477500
cell_e_series_in_front_of_primary_C       12.907900
cell_e_series_lower_plenum_C              10.618600
cell_e_series_midplate_C                  10.949900
cell_e_series_outside_ambient_C           15.325400
yankee_temperature                        15.146000
temptrax1_probe2                          14.800000
temptrax1_probe3                          15.200000
temptrax1_probe4                          15.200000
temptrax1_probe6                          14.700000
temptrax3_probe10                         15.200000
temptrax3_probe11                         15.400000
temptrax3_probe12                         15.200000
Name: 5502, 

In [ ]:
chk_data = normed_test_data[normed_test_data['f9'] > 0]
#chk.index
chk_test = test_labels['focus'][chk_data.index] 
chk_pred = models['focus'].predict(chk_data).flatten()
chk_diff = chk_test - chk_pred
chk_diff.std()

In [ ]:
chk_diff

In [ ]:
test_dataset.loc[3125]

In [ ]:
m = keras.models.load_model("/Users/tim/MMT/HALcoll/halcoll/data/tiltx_model.h5")

In [ ]:
m.evaluate(normed_test_data, test_labels['tiltx'])

In [ ]:
ttt = train_dataset['cell_e_series_chamber_ambient_C'] - train_dataset['yankee_temperature']
ttt.min()

In [ ]:
m = models['focus']

In [ ]:
m.summary()

In [31]:
test_labels['f9'].keys()

dict_keys(['focus', 'tiltx', 'tilty', 'transx', 'transy'])